In [34]:
import tensorflow as tf
from tensorflow.keras import layers

In [35]:
def resnet_block(inputs, filters, kernel_size=3, strides=1):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(inputs)
    shortcut = layers.BatchNormalization()(shortcut)
    x = layers.Add()([x, shortcut])
    x = layers.LeakyReLU()(x)
    return x

In [36]:
def build_model():
    # Fully Connected Neural Network 1
    input_fc1 = layers.Input(shape=(4,))
    x = layers.Dense(16)(input_fc1)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(64)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(256)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(64)(x)
    x = layers.Reshape((8, 8, 1))(x)
    x = layers.Softmax()(x)
    x = layers.Lambda(lambda x: x * 30.3)(x)
    output_fc1 = x

    # RESNET v1 1
    #input_resnet1 = layers.Input(shape=(8, 8, 1))
    input_resnet1 = output_fc1
    x = resnet_block(input_resnet1, filters=64)
    x = resnet_block(x, filters=64)
    output_resnet1 = x

    # RESNET v1 2
    #input_resnet2 = layers.Input(shape=(8, 8, 64))
    input_resnet2 = output_resnet1 
    x = resnet_block(input_resnet2, filters=64)
    x = resnet_block(x, filters=64)
    output_resnet2 = x

    # Fully Connected Neural Network 2
    #input_fc2 = layers.Input(shape=(32, 32, 64))
    input_fc2 = output_resnet2
    x = layers.Flatten()(input_fc2)
    x = layers.Dense(512)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(512)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dense(256)(x)
    x = layers.LeakyReLU()(x)
    output_fc2 = layers.Dense(72)(x)

    # Build the model
    model = tf.keras.Model(
        inputs=[input_fc1],
        outputs=[output_fc2]
    )
    return model


In [37]:
# Create the model
model = build_model()

# Print the model summary
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 dense_16 (Dense)               (None, 16)           80          ['input_9[0][0]']                
                                                                                                  
 leaky_re_lu_36 (LeakyReLU)     (None, 16)           0           ['dense_16[0][0]']               
                                                                                                  
 dense_17 (Dense)               (None, 64)           1088        ['leaky_re_lu_36[0][0]']         
                                                                                            

In [38]:
import numpy as np
from sklearn import preprocessing

# Specify the file path
file_path = 'inputx.dat'  # Path to the .dat file

# Read the file and extract the 2nd and 3rd columns
data = np.loadtxt(file_path)
input_1 = data[:, 1]
input_2 = data[:, 2]

# Split column_2 into two columns
columnx_1_new = []
columnx_2_new = []

for i in range(len(input_1)):
    if i % 2 == 0:
        columnx_1_new.append(input_1[i])
    else:
        columnx_2_new.append(input_1[i])

# Split column_2 into two columns
columny_1_new = []
columny_2_new = []

for i in range(len(input_2)):
    if i % 2 == 0:
        columny_1_new.append(input_2[i])
    else:
        columny_2_new.append(input_2[i])

input_1_n = preprocessing.normalize([columnx_1_new])
input_2_n = preprocessing.normalize([columny_1_new])
input_3_n = preprocessing.normalize([columnx_2_new])
input_4_n = preprocessing.normalize([columny_2_new])

# Reshape the normalized arrays
reshaped_column_1 = input_1_n.reshape((-1, 1))
reshaped_column_2 = input_2_n.reshape((-1, 1))
reshaped_column_3 = input_3_n.reshape((-1, 1))
reshaped_column_4 = input_4_n.reshape((-1, 1))


# Combine the columns into a single input array
x_data = np.concatenate((reshaped_column_1, reshaped_column_2, reshaped_column_3, reshaped_column_4), axis=1)
x_data = np.array(x_data)
# Print the arrays
#print("2nd Column:")
print(x_data)
#print("3rd Column:")
#print(input_2_n)


[[0.39640449 0.13960217 0.57199814 0.15102671]
 [0.47629808 0.06937963 0.01343947 0.36663901]
 [0.03326471 0.48048841 0.09347401 0.04499018]
 [0.08573504 0.40763221 0.17821131 0.24422197]
 [0.18612654 0.33937823 0.20420601 0.47207737]
 [0.23985387 0.26664292 0.28740184 0.14888868]
 [0.2696753  0.19161215 0.39990293 0.33459123]
 [0.30170048 0.11679254 0.50970212 0.52161131]
 [0.30682204 0.03939005 0.04280173 0.1693883 ]
 [0.32800488 0.44712591 0.16589173 0.34993151]
 [0.38716753 0.37491289 0.2424258  0.02998831]]


In [39]:

#import numpy as np

# Specify the file path
file_path = 'inputy.dat'  # Path to the .dat file

# Read the file and extract the 2nd and 3rd columns
data = np.loadtxt(file_path)
output_1 = data[:, 1]
output_2 = data[:, 2]


output_1_n = preprocessing.normalize([output_1])
output_2_n = preprocessing.normalize([output_2])

# Reshape the normalized arrays
reshapedy_column_2 = output_1_n.reshape((-1, 1))
reshapedy_column_3 = output_2_n.reshape((-1, 1))


# Combine the columns into a single input array
y_data_2 = np.concatenate((reshapedy_column_2, reshapedy_column_3), axis=1)

# Print the arrays
#print("2nd Column:")
#print(y_data)
#print("3rd Column:")
#print(output_2)

def convert_matrix(original_matrix, m):
    original_rows, original_cols = len(original_matrix), len(original_matrix[0])
    new_matrix = [[0] * m for _ in range(72)]  # Create an empty 72xm matrix
    
    for row in range(72):
        for col in range(m):
            original_row = row % original_rows
            original_col = col % original_cols
            new_matrix[row][col] = original_matrix[original_row][original_col]
    
    return new_matrix


y_data = convert_matrix(y_data_2, 72)
y_data = np.array(y_data)
print(y_data)

[[0.0145037  0.01549143 0.0145037  ... 0.01549143 0.0145037  0.01549143]
 [0.00693028 0.02522649 0.00693028 ... 0.02522649 0.00693028 0.02522649]
 [0.00026693 0.01052226 0.00026693 ... 0.01052226 0.00026693 0.01052226]
 ...
 [0.07262263 0.04813296 0.07262263 ... 0.04813296 0.07262263 0.04813296]
 [0.03403758 0.03772074 0.03403758 ... 0.03772074 0.03403758 0.03772074]
 [0.04995597 0.01407055 0.04995597 ... 0.01407055 0.04995597 0.01407055]]


In [42]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(x_data, y_data, batch_size=32, epochs=10, validation_split=0.2)

# Evaluate the model
#loss, mae = model.evaluate(x_test, [y_test] * 4)

# Make predictions
#predictions = model.predict(x_test)

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 0.0466 - accuracy: 0.0000e+00 - val_loss: 15.0713 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 161ms/step - loss: 4.9928 - accuracy: 0.0000e+00 - val_loss: 5.6411 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 131ms/step - loss: 0.6189 - accuracy: 0.0000e+00 - val_loss: 7.1401 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 158ms/step - loss: 0.5495 - accuracy: 0.0000e+00 - val_loss: 4.9089 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 131ms/step - loss: 0.2846 - accuracy: 0.0000e+00 - val_loss: 3.1515 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 120ms/step - loss: 0.1610 - accuracy: 0.0000e+00 - val_loss: 2.9009 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 132ms/step - loss: 0.1320 - accuracy: 0.0000e+00 - val_loss: 